In [1]:
#-*-coding:utf-8
import bs4
from urllib.request import urlopen
import urllib.request as request
import datetime as dt
import pandas as pd
import collections
from time import sleep

import re

In [2]:
def stock_info(stock_cd):
    url_float = 'https://companyinfo.stock.naver.com/company/c1010001.aspx?cmp_cd=' + str(stock_cd)
    
    #if '030200' == str(stock_cd) or '005930' == str(stock_cd):
    #    sleep(7)
    #    print(url_float)
    
    #proxy_support = request.ProxyHandler({"http": "http://168.219.61.252:8080/", "https": "http://168.219.61.252:8080/"})
    #opener = request.build_opener(proxy_support)

    source = urlopen(url_float, context = ssl._create_unverified_context()).read()
    soup = bs4.BeautifulSoup(source, 'html.parser')
    
    #if '030200' == str(stock_cd) or '005930' == str(stock_cd):
    #    print(source)
    #    print(soup)
    #    print(soup.find(id='cTB11'))
    try:
        tmp = soup.find(id='cTB11').find_all('tr')[6].td.text
        tmp = tmp.replace('\r','')
        tmp = tmp.replace('\n','')
        tmp = tmp.replace('\t','')
    except IndexError:
        print(url_float)
    
    tmp = re.split('/',tmp)

    outstanding = tmp[0].replace(',','')
    outstanding = outstanding.replace('주','')
    outstanding = outstanding.replace(' ','')
    outstanding = int(outstanding)

    floating = tmp[1].replace(' ','')
    floating = floating.replace('%','')
    floating = float(floating)
    
    name = soup.find(id='pArea').find('div').find('div').find('tr').find('td').find('span').text
    
    k50_outstanding[stock_cd] = outstanding
    k50_floating[stock_cd] = floating
    k50_name[stock_cd] = name

In [3]:
def top_rank_stocks(num, top_ranks, page_n=1, last_page=0):
    url_float = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page=' + str(page_n)

    #proxy_support = request.ProxyHandler({"http": "http://168.219.61.252:8080/", "https": "http://168.219.61.252:8080/"})
    #opener = request.build_opener(proxy_support)

    source = urlopen(url_float).read().decode('euc-kr','replace').encode('utf-8','replace')
    soup = bs4.BeautifulSoup(source, 'lxml')

    tmp = soup.find_all('a', href=True)

    for stock_name in tmp:
        if '/item/main.nhn?code' in stock_name['href']:
            #top_ranks.append(stock_name.text:(re.split('=',stock_name['href']))[1])
            prefix_matcher = list()
            if 'KINDEX' in stock_name.text or 'ARIRANG' in stock_name.text or 'KODEX' in stock_name.text or 'TIGER' in stock_name.text or 'KBSTAR' in stock_name.text or 'TRUE 코스피' in stock_name.text:
                continue
            
            if '우' in stock_name.text:
                #print("dddddddd",stock_name.text)                
                if '대우' in stock_name.text:
                    idx_num = stock_name.text.index('우')
                    prefix_matcher.append(stock_name.text[idx_num+1:])
                    if '우' in prefix_matcher[0]:
                        #print("ccccccc",prefix_matcher[0])
                        prefix_matcher[0] = stock_name.text[:idx_num+1]
                    else:
                        prefix_matcher[0] = stock_name.text                     
                    #print("bbbbbbbb",prefix_matcher[0])
                else:
                    #print(stock_name.text)
                    prefix_matcher = re.split('[0-9]*우',stock_name.text)
                    #print(prefix_matcher[0], prefix_matcher[0] in top_ranks)
                if prefix_matcher[0] in top_ranks:
                    #print("aaaaaaaa",stock_name.text)
                    continue
            #print(stock_name['href'], stock_name.text)
            top_ranks[stock_name.text] = (re.split('=',stock_name['href']))[1]

        if len(top_ranks) >= num:
            return top_ranks

    if last_page == 0:
        last_page = soup.find('td', class_='pgRR').find('a')['href']
        #마지막 주소 추출
        last_page = last_page.split('&')[-1]
        last_page = int(last_page.split('=')[-1])            

    if page_n < last_page:
        page_n = page_n + 1
        
        top_rank_stocks(num, top_ranks, page_n, last_page)
    
    return top_ranks

In [4]:
def date_time(d):
    d = str(d).replace('-','.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy,mm,dd)
    return this_date

def historical_index_naver_domestic(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date:
        start_date = date_time(start_date)
    else:
        start_date = dt.date.today()
        
    if end_date:
        end_date = date_time(end_date)
    else:
        end_date = dt.date.today()    
    
    if index_cd.isdigit() == True:
        naver_index = 'https://finance.naver.com/item/sise_day.nhn?code='+index_cd+'&page='+str(page_n)
    else:
        naver_index = 'https://finance.naver.com/sise/sise_index_day.nhn?code='+index_cd+'&page='+str(page_n)

    #proxy_support = request.ProxyHandler({"http": "http://168.219.61.252:8080/", "https": "http://168.219.61.252:8080/"})
    #opener = request.build_opener(proxy_support)

    source = urlopen(naver_index).read()             #지정한 페이지 읽기
    source = bs4.BeautifulSoup(source, 'lxml')       #BeautifulSoup으로 태그별 코드 분류
    

    if index_cd.isdigit() == True:
        dates = source.find_all('span', class_='tah p10 gray03')
        prices = source.find_all('td', class_='num')
        prices_temp = prices.copy()
        prices_temp.clear()
        #print(type(source),type(prices))
        for words in prices:
            prices_temp += words.find_all('span')
        prices = prices_temp
    else:
        dates = source.find_all('td', class_='date')
        prices = source.find_all('td', class_='number_1')
    
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            this_date = dates[n].text
            this_date = date_time(this_date)
            
            
            if this_date <= end_date and this_date >= start_date:
                temp_prices = dict()               
                if index_cd.isdigit() == True:
                    temp_prices['종가'] = prices[n*6].text #종가
                    temp_prices['종가'] = float(temp_prices['종가'].replace(',',''))
                    temp_prices['시가'] = prices[n*6 + 2].text #시가
                    temp_prices['시가'] = float(temp_prices['시가'].replace(',',''))
                    if temp_prices['시가'] == 0:
                        temp_prices['시가'] = temp_prices['종가']
                    temp_prices['고가'] = prices[n*6 + 3].text #고가
                    temp_prices['고가'] = float(temp_prices['고가'].replace(',',''))
                    if temp_prices['고가'] == 0:
                        temp_prices['고가'] = temp_prices['종가']
                    temp_prices['저가'] = prices[n*6 + 4].text #고가
                    temp_prices['저가'] = float(temp_prices['저가'].replace(',',''))
                    if temp_prices['저가'] == 0:
                        temp_prices['저가'] = temp_prices['종가']
                    this_close = temp_prices
                    #print(this_close)
                else:
                    this_close = prices[n*4].text
                    this_close = this_close.replace(',','')
                    this_close = float(this_close)

                historical_prices[this_date] = this_close
                
            elif this_date < start_date:
                return historical_prices
    
    if last_page == 0:
        #print(naver_index)
        #print(source)
        #print(source.find('td', class_='pgRR'))
        if (source.find('td', class_='pgRR') != None):
            last_page = source.find('td', class_='pgRR').find('a')['href']
            #마지막 주소 추출
            last_page = last_page.split('&')[-1]
            last_page = int(last_page.split('=')[-1])
        
    if page_n < last_page:
        page_n = page_n + 1
        historical_index_naver_domestic(index_cd, start_date, end_date, page_n, last_page)
    
    return historical_prices

In [5]:
top_ranks = collections.OrderedDict()
k50_outstanding = dict()
k50_floating = dict()
k50_name = dict()

#Chnaeg the value if you want to get more 100 -> ?
top_ranks = top_rank_stocks(10, top_ranks)
#print(top_ranks.values())
k50_component = top_ranks.values()
for stock_cd in k50_component:
    #print(stock_cd)
    stock_info(stock_cd)
    

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:852)>

In [ ]:
k50_historical_prices = dict()
date1 = dt.date.today()
date2 = dt.timedelta(days=366)
start_day = date1 - date2
for stock_cd in k50_component:
    
    historical_prices = dict()
    start_date = str(start_day.year) + '-' + str(start_day.month) + '-' + str(start_day.day)
    end_date = str(date1.year) + '-' + str(date1.month) + '-' + str(date1.day)
    historical_index_naver_domestic(stock_cd, start_date, end_date)
    
    if '삼성전자' in top_ranks and top_ranks['삼성전자'] == stock_cd:
        tmp_historical_prices = dict()
        divideDate = dt.date(2018,5,4)
        tmp_historical_prices = dict()
        for key, value in historical_prices.items():
            if type(value) is dict:
                tmp_prices = dict()
                for inner_key, inner_value in value.items():
                    if date_time(key) < divideDate:
                        tmp_prices[inner_key] = float(inner_value)/50
                    else:
                        tmp_prices[inner_key] = float(inner_value)
                tmp_historical_prices[key] = tmp_prices
            else:
                if date_time(key) < divideDate:
                    tmp_historical_prices[key] = float(value)/50
                else:
                    tmp_historical_prices[key] = float(value)

        k50_historical_prices[stock_cd] = tmp_historical_prices
    else:
        k50_historical_prices[stock_cd] = historical_prices
    
k50_historical_prices = pd.DataFrame(k50_historical_prices)

In [ ]:
k50_historical_prices = k50_historical_prices.fillna(method='ffill')
if k50_historical_prices.isnull().values.any():
    k50_historical_prices = k50_historical_prices.fillna(method='bfill')
#k50_historical_prices.head(5)

In [ ]:
k50_min = dict()
K50_min_date = dict()
K50_gap = dict()
today = dt.date.today()
for key, values in k50_historical_prices.items():
    #print(key, values[dt.date(2018,4,30)])
    key_name = str()
    search_list = values[:-1]
    key_min_date = min(search_list.keys(), key=(lambda k: search_list[k]['저가']))
    for name in top_ranks:
        if top_ranks[name] == key:
            key_name = name
            break
    #print(key, search_list[dt.date(2018,4,30)])
    #print(key, search_list[key_min_date])
    #print(key_min_date)
    #print(type(key_min_date))
    #print(values[key_min_date])
    #print(values[key_min_date]['저가'])
    k50_min[key]=values[key_min_date]['저가']
    K50_min_date[key]=key_min_date
    K50_gap[key]=values[-2]['종가'] - k50_min[key]

In [ ]:
Today_price = dict((key, value['종가']) for key, value in  k50_historical_prices.iloc[-1].items())
Yesterday_price = dict((key, value['종가']) for key, value in  k50_historical_prices.iloc[-2].items())
volume = dict()
gap_percentage = dict()

for index, content in enumerate(top_ranks):
    volume[top_ranks[content]] = int(index + 1)
    gap_percentage[top_ranks[content]] = float("%2.1f"%(((Yesterday_price[top_ranks[content]]/k50_min[top_ranks[content]])-1)*100))

tmp = {'Outstanding':k50_outstanding, \
      'Floating':k50_floating, \
      'Min_52_Date':K50_min_date, \
      'Min_52':k50_min, \
      'Yesterday_price':Yesterday_price,\
      'Today_price':Today_price,\
      'Gap':K50_gap,\
      'Gap_Percentage':gap_percentage,\
      'Volume':volume,\
      'Name':k50_name}
k50_info = pd.DataFrame(tmp,columns=['Name', 'Min_52_Date', 'Min_52', 'Yesterday_price', 'Gap', 'Gap_Percentage', 'Today_price', 'Volume'])

In [ ]:
from IPython.display import display
pd.options.display.max_rows = None

k50_info = k50_info.sort_values(["Gap_Percentage"], ascending=[True])
display(k50_info)